# Machine Learning with PyTorch and Scikit-Learn  
## Chapter 8: Applying Machine Learning to Sentiment Analysis


- We will delve into a subfield of natural language processing (NLP) called sentiment analysis

The topics that we will cover in this chapter include the following: 

• Cleaning and preparing text data 

• Building feature vectors from text documents 

• Training a machine learning model to classify positive and negative movie reviews 

• Working with large text datasets using out-of-core learning 

• Inferring topics from document collections for categorization

# Preparing the IMDb movie review data for text processing 

- Opinion mining

- The movie review dataset consists of 50,000 polar movie reviews that are labeled as either positive or negative; here, positive means that a movie was rated with more than six stars on IMDb, and negative means that a movie was rated with fewer than five stars on IMDb

![](2022-10-23-10-56-16.png)

## Obtaining the IMDb movie review dataset

- The IMDB movie review set can be downloaded from [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/).

- After downloading the dataset, decompress the files.

- A) If you are working with Linux or MacOS X, open a new terminal windowm `cd` into the download directory and execute 

`tar -zxf aclImdb_v1.tar.gz`

- B) If you are working with Windows, download an archiver such as [7Zip](http://www.7-zip.org) to extract the files from the download archive.

**Optional code to download and unzip the dataset via Python:**

In [2]:
import os
import sys
import tarfile
import time
import urllib.request

source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'

if os.path.exists(target):
    os.remove(target)

def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write(f'\r{int(percent)}% | {progress_size / (1024.**2):.2f} MB '
                     f'| {speed:.2f} MB/s | {duration:.2f} sec elapsed')
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

[Don’t Use Python OS Library Any More When Pathlib Can Do:](https://towardsdatascience.com/dont-use-python-os-library-any-more-when-pathlib-can-do-141fefb6bdb5)


In [3]:
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

## Preprocessing the movie dataset into more convenient format

- Having successfully extracted the dataset, we will now assemble the individual text documents from the decompressed download archive into a single CSV file.

 - In the following code section, we will be reading the movie reviews into a pandas DataFrame object, which can take up to 10 minutes on a standard desktop computer.

In [22]:
#Install pyprind by uncommenting the next code cell.
#!pip install pyprind

> The PyPrind (Python Progress Indicator) module provides a progress bar and a percentage indicator object that let you track the progress of a loop structure or other iterative computation

Find more here: https://github.com/rasbt/pyprind/blob/master/README.md

In [4]:
import pyprind

n = 3
timesleep=2

for i in pyprind.prog_bar(range(n), stream=sys.stderr):
    time.sleep(timesleep) # your computation here

0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


In [5]:
import pyprind
import os
import sys
import pandas as pd

# change the `basepath` to the directory of the
# unzipped movie dataset

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

/var/folders/1h/b7ng0kgj3w78mg7n8k7q7rch0000gn/T/ipykernel_46718/3148269608.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt, labels[l]]],


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


In [6]:
df

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
...,...,...
49995,"Towards the end of the movie, I felt it was to...",0
49996,This is the kind of movie that my enemies cont...,0
49997,I saw 'Descent' last night at the Stockholm Fi...,0
49998,Some films that you pick up for a pound turn o...,0


## Shuffling the DataFrame:

In [7]:
df.index

RangeIndex(start=0, stop=50000, step=1)

In [10]:
import numpy as np
np.random.permutation(df.index)

array([18105, 48231, 39414, ..., 12981, 47584, 10118])

In [11]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [12]:
df

,review,sentiment
11841,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
19602,OK... so... I really like Kris Kristofferson a...,0
45519,"***SPOILER*** Do not read this, if you think a...",0
25747,hi for all the people who have seen this wonde...,1
42642,"I recently bought the DVD, forgetting just how...",0
...,...,...
21243,"OK, lets start with the best. the building. al...",0
45891,The British 'heritage film' industry is out of...,0
42613,I don't even know where to begin on this one. ...,0
43567,Richard Tyler is a little boy who is scared of...,0


Optional: Saving the assembled data as CSV file:

In [13]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

let’s quickly confirm that we have successfully saved the data in the right format by reading in the CSV and printing an excerpt of the first three examples:

In [14]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

# the following is necessary on some computers:


df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [15]:
df_old = df.copy()

In [16]:
df.shape

(50000, 2)

---

### Note

If you have problems with creating the `movie_data.csv`, you can find a download a zip archive at 
https://github.com/rasbt/machine-learning-book/tree/main/ch08/

---

## Vectorization:  Transforming documents into feature vectors

- Vectorization is jargon for a classic approach of converting input data from its raw format (i.e. text ) into vectors of real numbers which is the format that ML models support. 

- In Machine Learning, vectorization is a step in feature extraction. The idea is to get some distinct features out of the text for the model to train on, by converting text to numerical vectors.

![](2022-10-23-14-08-20.png)

### Vectorization techniques

- Bag of Words
- TF-IDF
- Word2Vec
- GloVe
- FastText







Read more: https://neptune.ai/blog/vectorization-techniques-in-nlp-guide

## 1. Introducing the bag-of-words model

In this section, we will introduce the bag-of-words model, which allows us to represent text as numerical feature vectors. 


The idea behind bag-of-words is quite simple and can be summarized as follows:

- We create a vocabulary of unique tokens—for example, words—from the entire set of documents. 


- We construct a feature vector from each document that contains the counts of how often each word occurs in the particular document.

![](2022-10-23-12-10-07.png)

> Since the unique words in each document represent only a small subset of all the words in the bag-ofwords vocabulary, the feature vectors will mostly consist of zeros, which is why we call them sparse.

> To construct a bag-of-words model based on the word counts in the respective documents, we can use the CountVectorizer class implemented in scikit-learn.

By calling the `fit_transform` method on CountVectorizer, we can constructed the vocabulary of the bag-of-words model and transform the individual text documents into sparse feature vectors.

1. The sun is shining
   
2. The weather is sweet
   
3. The sun is shining, the weather is sweet, and one and one is two


In [17]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])

bag = count.fit_transform(docs)

Now let us print the contents of the vocabulary to get a better understanding of the underlying concepts:

In [18]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


As we can see from executing the preceding command, the vocabulary is stored in a Python dictionary, which maps the unique words that are mapped to integer indices. Next let us print the feature vectors that we just created:

In [19]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Each index position in the feature vectors shown here corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. 


For example, the  1st feature at index position 0 resembles the count of the word and, which only occurs in the last document, and the word is at index position 1 (the 2nd feature in the document vectors) occurs in all three sentences. 
  
  
---
> Those values in the feature vectors are also called the raw term frequencies: *tf (t,d)*—the number of times a term t occurs in a document *d*.
---


## N-gram models

- The sequence of items in the bag-of-words model that we just created is also called the 1-gram or unigram model—each item or token in the vocabulary represents a single word. 

- More generally, the contiguous sequences of items in NLP—words, letters, or symbols—are also called n-grams.


![](2022-10-23-12-30-26.png)



---
> The choice of the number, n, in the n-gram model depends on the particular application; for example, in sentiment analysis, we might want to consider not only single words but also word pairs or triplets, which can capture the meaning of a sentence better than single words.
---

---
>  For example, the phrase The car is not <font color='green'>not good</font> . **not good** is a negative expression, whereas the phrase <font The car is not color='yellow'>not bad</font> is a positive expression.
---

<br>

In [108]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count2 = CountVectorizer(ngram_range=(1, 2))

#count2 = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag2 = count2.fit_transform(docs)

In [109]:
print(count2.vocabulary_)

{'the': 15, 'sun': 11, 'is': 2, 'shining': 9, 'the sun': 16, 'sun is': 12, 'is shining': 3, 'weather': 19, 'sweet': 13, 'the weather': 17, 'weather is': 20, 'is sweet': 4, 'and': 0, 'one': 6, 'two': 18, 'shining the': 10, 'sweet and': 14, 'and one': 1, 'one and': 7, 'one is': 8, 'is two': 5}


## 2. Assessing word relevancy via term frequency-inverse document frequency (TF-IDF)

In [22]:
np.set_printoptions(precision=2)

- When we are analyzing text data, we often encounter words that occur across multiple documents from both classes. Those frequently occurring words typically don't contain useful or discriminatory information.

- we will learn about a useful technique called term **frequency-inverse document frequency(tf-idf)** that can be used to downweight those frequently occurring words in the feature vectors. 
  
- The tf-idf can be de ned as the product of the term frequency and the inverse document frequency:

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

Here the tf(t, d) is the term frequency that we introduced in the previous section,
and the inverse document frequency *idf(t, d)* can be calculated as:

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

where $n_d$ is the total number of documents, and *df(d, t)* is the number of documents *d* that contain the term *t*. Note that adding the constant 1 to the denominator is optional and serves the purpose of assigning a non-zero value to terms that occur in all training examples; the log is used to ensure that low document frequencies are not given too much weight.

Scikit-learn implements yet another transformer, the `TfidfTransformer`, that takes the raw term frequencies from `CountVectorizer` as input and transforms them into tf-idfs:

In [23]:
docs

array(['The sun is shining', 'The weather is sweet',
       'The sun is shining, the weather is sweet, and one and one is two'],
      dtype='<U64')

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


---
> As you saw in the previous subsection, the word 'is' had the largest term frequency in the third document, being the most frequently occurring word. However, after transforming the same feature vector into tf-idfs, the word 'is' is now associated with a relatively small tf-idf (0.45) in the third document, since it is also present in the first and second document and thus is unlikely to contain any useful discriminatory information
---

> While it is also more typical to normalize the raw term frequencies before calculating the tf-idfs, the TfidfTransformer class normalizes the tf-idfs directly. By default (norm='l2')

> To make sure that we understand how TfidfTransformer works, check the example from the book, where they calculate the tf-idf manually.


## Why TF-IDF can be a better representation of text documents than bag of words?

- Bag of Words just creates a set of vectors containing the count of word occurrences in the document (reviews), while the TF-IDF model contains information on the more important words and the less important ones as well

- The bag-of-words model is commonly used in methods of document classification where the (frequency of) occurrence of each word is used as a feature for training a classifier.

- The resultant vectors will be of large dimension and will contain far too many null values resulting in sparse vectors.

---
> Apart from resulting in sparse representations, Bag of Words does a poor job in making sense of text data
---
 

## Cleaning text data

<img src="2022-10-23-12-35-59.png" width="500" height="400" align="left" style="margin-right: 20px;">
<img src="2022-10-23-12-36-28.png" width="300" height="400" align="right" style="margin-right: 40px;">

- The first important step—before we build our bag-of-words model—is to clean the text data by stripping it of all unwanted characters.

In [25]:
df.loc[0, 'review'][-80:]

'in the dramatization. My vote is seven.<br /><br />Title (Brazil): Not Available'

we will use Python’s regular expression (regex) library, re, as shown here:

In [26]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [27]:
exampletext = 'foo bar\t baz \tqux'

preprocessor(exampletext)

'foo bar baz qux'

In [28]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [29]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [30]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

let’s now apply our preprocessor function to all the movie reviews in our DataFrame

In [31]:
df

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
...,...,...
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0


In [32]:
df['review'] = df['review'].apply(preprocessor)

In [33]:
df.head(10)

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0
5,leave it to braik to put on a good show finall...,1
6,nathan detroit frank sinatra is the manager of...,1
7,to understand crash course in the right contex...,1
8,i ve been impressed with chavez s stance again...,1
9,this movie is directed by renny harlin the fin...,1


In [34]:
df_old.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


---
> In the context of sentiment analysis, it is a simplifying assumption that the letter case does not contain information that is relevant for sentiment analysis. However, in other applications, such as topic modeling, it might be important to preserve the letter case.
---

Learning regular expression come with a steep learning curve: 

Resource to learn about Python regular expressions: 

1. https://docs.python.org/3/howto/regex.html
2. https://developers.google.com/edu/python/regular-expressions
3. https://realpython.com/regex-python/




## Tokenization: Processing documents into tokens 

- After successfully preparing the movie review dataset, we now need to think about how to split the text corpora into individual elements. 

- One way to tokenize documents is to split them into individual words by splitting the cleaned documents at their whitespace characters

In [35]:
def tokenizer(text): 
    return text.split() 

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

But, running and run are same words? 

In the context of tokenization, another useful technique is word stemming, which is the process of transforming a word into its root form. It allows us to map related words to the same stem

> To install the NLTK, you can simply execute 

In [36]:
#!conda install nltk or pip install nltk. # select conda or pip

In [37]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [38]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [39]:
tokenizer_porter('birds bird')

['bird', 'bird']

In [40]:
tokenizer_porter('run ran running')

['run', 'ran', 'run']

## Stopwords Removal

- Stopwords are words that are very common in a language and that do not contain any useful information for the task at hand.

- Examples of stopwords are : the, a, an, and, but, is, are, etc.

 
 ![](2022-10-23-13-22-36.png) 

- Removing stop words can be useful if we are working with raw or normalized term frequencies rather than tf-idfs, which already downweight the frequently occurring words.

In [44]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shmuhammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
from nltk.corpus import stopwords

print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [50]:
print(stopwords.words('german'))


['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anderr', 'anders', 'auch', 'auf', 'aus', 'bei', 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass', 'daß', 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe', 'dazu', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich', 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein', 'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger', 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es', 'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer', 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier', 'hin', 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euc

In [51]:
]from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot')
 if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

We now understand how to clean and tokenize text data, and we have also learned how to remove stopwords. We will train a logistic regression model on the movie review dataset in the next section.

# Training a logistic regression model for document classification

Strip HTML and punctuation to speed up the GridSearch later:

In [52]:
df

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0
...,...,...
49995,ok lets start with the best the building altho...,0
49996,the british heritage film industry is out of c...,0
49997,i don t even know where to begin on this one i...,0
49998,richard tyler is a little boy who is scared of...,0


Splittng the dataset into train and test

In [105]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

## Using Sklearn

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV


# we replaced  CountVectorizer  and TfidfTransformer from the previous subsection with 
# TfidfVectorizer**, which combines CountVectorizer with the TfidfTransformer

tfidf = TfidfVectorizer(strip_accents=None, # Equivalent to CountVectorizer followed by TfidfTransformer.
                        lowercase=False,
                        preprocessor=None)

"""
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
"""

small_param_grid = [{'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [None], # no stop words
                     'vect__tokenizer': [tokenizer, tokenizer_porter],
                     'clf__penalty': ['l2'],
                     'clf__C': [1.0, 10.0]},
                     
                     {'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [stop, None], # with stop words
                     'vect__tokenizer': [tokenizer], # no stemming
                     'vect__use_idf':[False], # no tf-idf
                     'vect__norm':[None], # no normalization
                     'clf__penalty': ['l2'], #use l2 penalty
                  'clf__C': [1.0, 10.0]},
              ]

# for the logistic regression classifier itself, we trained models using L2 regularization via the penalty parameter and compared 
# different regularization strengths by defining a range of values for the inverse-regularization parameter C.

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver='liblinear'))])
#LIBLINEAR solver as it can perform better than the default choice ('lbfgs') for relatively large datasets.

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1) # -1 means use all processors, and speed up the grid search.

> As an optional exercise, you are also encouraged to add L1 regularization to the parameter grid by changing 'clf__penalty': ['l2'] to 'clf__penalty': ['l2', 'l1'].

**Important Note about `n_jobs`**
> some Windows users reported issues when running the previous code with the `n_jobs=-1` setting related to pickling the tokenizer and tokenizer_porter functions for multiprocessing on Windows. Another workaround would be to replace those two functions, `[tokenizer, tokenizer_porter]`, with `[str.split]`. However, note that the replacement by the simple `str.split` would not support stemming.

In [57]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x29b5bd040>,
                                              <function tokenizer_porter at 0x29b5bd0d0>]},
                         {'...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x29b5bd040>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=1)

After the grid search has finished, we can print the best parameter set:

In [58]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')


Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x29b5bd040>}
CV Accuracy: 0.897


> As you can see in the preceding output, we obtained the best grid search results using the regular tokenizer without Porter stemming, no stop word library, and tf-idfs in combination with a logistic regression classifier that uses L2-regularization with the regularization strength C of 10.0

- Using the best model from this grid search, let’s print the average 5-fold cross-validation accuracy scores on the training dataset and the classification accuracy on the test dataset:

In [60]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

CV Accuracy: 0.897


In [59]:
clf = gs_lr_tfidf.best_estimator_ # best estimator to use for prediction
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

Test Accuracy: 0.899


<hr>
<hr>

####  Start comment:
    
Please note that `gs_lr_tfidf.best_score_` is the average k-fold cross-validation score. I.e., if we have a `GridSearchCV` object with 5-fold cross-validation (like the one above), the `best_score_` attribute returns the average score over the 5-folds of the best model. To illustrate this with an example:

In [61]:
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(n_splits=5, shuffle=False).split(X, y))
    
lr = LogisticRegression()
cross_val_score(lr, X, y, cv=cv5_idx)

array([0.6, 0.4, 0.6, 0.2, 0.6])

By executing the code above, we created a simple data set of random integers that shall represent our class labels. Next, we fed the indices of 5 cross-validation folds (`cv5_idx`) to the `cross_val_score` scorer, which returned 5 accuracy scores -- these are the 5 accuracy values for the 5 test folds.  

Next, let us use the `GridSearchCV` object and feed it the same 5 cross-validation sets (via the pre-generated `cv5_idx` indices):

In [62]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()
gs = GridSearchCV(lr, {}, cv=cv5_idx, verbose=3).fit(X, y) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..................................., score=0.600 total time=   0.0s
[CV 2/5] END ..................................., score=0.400 total time=   0.0s
[CV 3/5] END ..................................., score=0.600 total time=   0.0s
[CV 4/5] END ..................................., score=0.200 total time=   0.0s
[CV 5/5] END ..................................., score=0.600 total time=   0.0s


As we can see, the scores for the 5 folds are exactly the same as the ones from `cross_val_score` earlier.

Now, the best_score_ attribute of the `GridSearchCV` object, which becomes available after `fit`ting, returns the average accuracy score of the best model:

In [63]:
gs.best_score_

0.48

As we can see, the result above is consistent with the average score computed the `cross_val_score`.

In [64]:
lr = LogisticRegression()
cross_val_score(lr, X, y, cv=cv5_idx).mean()

0.48

#### End comment.

<hr>
<hr>

# Working with bigger data - online algorithms and out-of-core learning

- It is computationally quite expensive to construct the feature vectors for the 50,000-movie review dataset during a grid search. 

- In many real-world applications, it is not uncommon to work with even larger datasets that can exceed our computer’s memory.

- Since not everyone has access to supercomputer facilities, we will now apply a technique called out-ofcore learning, which allows us to work with such large datasets by fitting the classifier incrementally on smaller batches of a dataset.

![](2022-10-29-13-20-31.png)

> Out-of-core leanring refers to the machine learning algorithms working with data cannot fit into the memory of a single machine, but that can easily fit into some data storage such as local hard disk or web repository. Reade more : https://xlearn-doc.readthedocs.io/en/latest/large/

In [ ]:
# This cell is not contained in the book but
# added for convenience so that the notebook
# can be executed starting here, without
# executing prior code in this notebook

import os
import gzip


if not os.path.isfile('movie_data.csv'):
    if not os.path.isfile('movie_data.csv.gz'):
        print('Please place a copy of the movie_data.csv.gz'
              'in this directory. You can obtain it by'
              'a) executing the code in the beginning of this'
              'notebook or b) by downloading it from GitHub:'
              'https://github.com/rasbt/machine-learning-book/'
              'blob/main/ch08/movie_data.csv.gz')
    else:
        with gzip.open('movie_data.csv.gz', 'rb') as in_f, \
                open('movie_data.csv', 'wb') as out_f:
            out_f.write(in_f.read())

In [85]:
import numpy as np
import re
from nltk.corpus import stopwords


stop = stopwords.words('english')

# we will define a tokenizer function that cleans the unprocess
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

#Next, we will define a generator function, stream_docs, that reads in and returns one document at a time:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label # generator function, so we can iterate over it using next()

To verify that our stream_docs function works correctly, let’s read in the first document from the movie_data.csv file, which should return a tuple consisting of the review text as well as the corresponding class label:

In [77]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

We will now define a function, `get_minibatch`, that will take a document stream from the stream_docs function and return a particular number of documents specified by the size parameter:

In [86]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

---
> Unfortunately, we can’t use CountVectorizer for out-of-core learning since it requires holding the complete vocabulary in memory. Also, TfidfVectorizer needs to keep all the feature vectors of the training dataset in memory to calculate the inverse document frequencies. However, another useful vectorizer for text processing implemented in scikit-learn is HashingVectorizer. HashingVectorizer is data-independent and makes use of the hashing trick via the 32-bit MurmurHash3 function by Austin Appleby (you can find more information about MurmurHash at https://en.wikipedia.org/wiki/MurmurHash). 
---

> The hashing trick is a clever way to reduce the dimensionality of the feature vectors by mapping the tokens onto integers in a fixed range. The advantage of this approach is that we can use a much smaller feature vector than the one that we would get with CountVectorizer or TfidfVectorizer. 

In [87]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

- Using the preceding code, we initialized HashingVectorizer with our tokenizer function and set the number of features to 2**21. 

In [92]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

# we reinitialized a logistic regression classifier by setting the loss parameter of SGDClassifier to 'log'. Note that by choosing a large number of features in HashingVectorizer, we reduce the chance of causing hash collisions, 
# but we also increase the number of coefficients in our logistic regression model.
clf = SGDClassifier(loss='log_loss', random_state=1) # udpate loss to log_loss


doc_stream = stream_docs(path='movie_data.csv')

> This estimator implements regularized linear models with stochastic
gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate). SGD allows minibatch (online/out-of-core) learning via the partial_fit method. For best results using the default learning rate schedule, the data should have zero mean and unit variance.

Now comes the really interesting part—having set up all the complementary functions, we can start the out-of-core learning using the following code:

In [93]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

> we made use of the PyPrind package to estimate the progress of our learning algorithm. We initialized the progress bar object with 45 iterations and, in the following for loop, we iterated over 45 mini-batches of documents where each mini-batch consists of 1,000 documents. Having completed the incremental learning process, we will use the last 5,000 documents to evaluate the performance of our model:

In [94]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

Accuracy: 0.868


---
> As you can see, the accuracy of the model is approximately 87 percent, slightly below the accuracy that we achieved in the previous section using the grid search for hyperparameter tuning. However, out-of-core learning is very memory efficient, and it took less than a minute to complete.
---

Finally, we can use the last 5,000 documents to update our model:

In [96]:
clf = clf.partial_fit(X_test, y_test)

## Modern Approach to Bag of Words

> A more modern alternative to the bag-of-words model is word2vec, an algorithm that Google released in 2013 (Efficient Estimation of Word Representations in Vector Space by T. Mikolov, K. Chen, G. Corrado, and J. Dean, https://arxiv.org/abs/1301.3781).

# Topic modeling

> Topic modeling describes the broad task of assigning topics to unlabeled text documents. For example, a typical application is the categorization of documents in a large text corpus of newspaper articles, we can assign category labels to those articles, for example, sports, finance, world news, politics, and local news

![](2022-10-30-07-32-44.png)

### Decomposing text documents with Latent Dirichlet Allocation (LDA)

Many approaches for topic modelling, we present popular approach called LDA (Latent Dirichlet Allocation)

> LDA is a generative probabilistic model that tries to find groups of words that appear frequently together across different documents. The input to an LDA is the bag-of-words model that we discussed earlier in this chapter.

- These frequently appearing words represent our topics, assuming that each document is a mixture of different words.
-  Given a bag-of-words matrix as input, LDA decomposes it into two new matrices:
   
   -  A document-to-topic matrix 
   
   -  A word-to-topic matrix

> LDA decomposes the bag-of-words matrix in such a way that if we multiply those two matrices together, we will be able to reproduce the input, the bag-of-words matrix, with the lowest possible error. 

- In practice, we are interested in those topics that LDA found in the bag-of-words matrix. 

- The only downside may be that we must define the number of topics beforehand—the number of topics is a hyperparameter of LDA that has to be specified manually.

### Review of Bag-of-Words

In [173]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])

bag = count.fit_transform(docs) # fit transform is the same as fit and transform

In [144]:
count.get_feature_names_out()

array(['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two',
       'weather'], dtype=object)

In [174]:
bag.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]])

### Latent Dirichlet Allocation with scikit-learn

- We will use the LatentDirichletAllocation in scikit-learn to decompose the movie review dataset and categorize it into different 10 topics


In [146]:
import pandas as pd

df1 = pd.read_csv('movie_data.csv', encoding='utf-8')

In [147]:
df1.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [148]:
# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})

In [149]:
df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


> Create Bag-of-words using CountVectorizer

We are going to use the already familiar CountVectorizer to create the bag-of-words matrix as input to the LDA.

In [181]:
#Convert a collection of text documents to a matrix of token counts.
from sklearn.feature_extraction.text import CountVectorizer # count vectorizer class

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values) 
#This is equivalent to fit followed by transform, but more efficiently implemented.

In [183]:
X

<50000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 2780504 stored elements in Compressed Sparse Row format>

---
> Notice that we set the maximum document frequency of words to be considered to 10 percent (max_ df=.1) to exclude words that occur too frequently across documents

The rationale behind the removal of frequently occurring words is that these might be common words appearing across all documents that are, therefore, less likely to be associated with a specific topic category of a given document. 


---
> Also, we limited the number of words to be considered to the most frequently occurring 5,000 words (max_features=5000), to limit the dimensionality of this dataset to improve the inference performed by LDA

Note: both **max_df=.1** and **max_features=5000** are hyperparameter values chosen arbitrarily, and readers are encouraged to tune them while comparing the results

In [182]:
count.get_feature_names_out()

array(['00', '000', '100', ..., 'zombie', 'zombies', 'zone'], dtype=object)

In [184]:
count.get_feature_names_out()[:30]

array(['00', '000', '100', '11', '12', '13', '13th', '14', '15', '16',
       '17', '18', '1930', '1930s', '1940', '1950', '1950s', '1960',
       '1960s', '1968', '1969', '1970', '1970s', '1971', '1972', '1973',
       '1978', '1979', '1980', '1980s'], dtype=object)

In [185]:
count.get_feature_names_out()[:-30]

array(['00', '000', '100', ..., 'wouldn', 'wound', 'wounded'],
      dtype=object)

In [186]:
count.get_feature_names_out()[-30:]

array(['wow', 'wrap', 'wrapped', 'wreck', 'wrestling', 'write', 'writer',
       'writers', 'writes', 'writing', 'written', 'wrong', 'wrote', 'wwe',
       'wwii', 'ya', 'yawn', 'yeah', 'year', 'yelling', 'yellow', 'yes',
       'yesterday', 'york', 'younger', 'youth', 'zero', 'zombie',
       'zombies', 'zone'], dtype=object)

In [161]:
X

<50000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 2780504 stored elements in Compressed Sparse Row format>

In [187]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

> Let us fit LatentDirichletAllocation estimator to the bag-of-words matrix and infer the 10 different topics from the documents (note that the model fitting can take up to 5 minutes or more on a laptop or standard desktop computer

In [167]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X) # fit and transform in one step 

> By setting learning_method='batch', we let the lda estimator do its estimation based on all available training data (the bag-of-words matrix) in one iteration, which is slower than the alternative 'online' learning method, but can lead to more accurate results (setting learning_method='online' is analogous to online or mini-batch learning. 

> The default value is 'online' in scikit-learn 0.18 and later, and 'batch' in earlier versions)

In [ ]:
import sklearn
sklearn.show_versions()

After fitting the LDA, we now have access to the components_ attribute of the lda instance, which stores a matrix containing the word importance (here, 5000) for each of the 10 topics in increasing order

In [170]:
lda.components_.shape

(10, 5000)

To analyze the results, let’s print the five most important words for each of the 10 topics. Note that the word importance values are ranked in increasing order. Thus, to print the top five words, we need to sort the topic array in reverse order:

In [171]:
n_top_words = 5
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


Based on reading the 5 most important words for each topic, we may guess that the LDA identified the following topics:
    
1. Generally bad movies (not really a topic category)
2. Movies about families
3. War movies
4. Art movies
5. Crime movies
6. Horror movies
7. Comedies
8. Movies somehow related to TV shows
9. Movies based on books
10. Action movies

To confirm that the categories make sense based on the reviews, let's plot 5 movies from the horror movie category (category 6 at index position 5):

In [172]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nHorror movie #{(iter_idx + 1)}:')
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


Using the preceeding code example, we printed the first 300 characters from the top 3 horror movies and indeed, we can see that the reviews -- even though we don't know which exact movie they belong to -- sound like reviews of horror movies, indeed. (However, one might argue that movie #2 could also belong to topic category 1.)

<br>
<br>

# Summary

...

---

Readers may ignore the next cell.

In [ ]:
! python ../.convert_notebook_to_script.py --input ch08.ipynb --output ch08.py

python: can't open file '/Users/shmuhammad/Document/VSCode/bookclub/ml-with-pytorch-sklearn/chapters/ch08/../.convert_notebook_to_script.py': [Errno 2] No such file or directory
